We need to get from mapped bam file to single fast5s:

Input is:
* mapped bam file.
* multifast5 of fastqs mapped to the reference

Output:
* folder with single fast5s that are mapped

What we need to do:
* get ID of all mapped fastqs
* extract them in batches
* make them single fast5s

In [1]:
import os

### INPUTS

### *****Define directories before running on different sampes*****

In [2]:
###define input directories here 
BAMIN_DIR = '../../analyses/mapping/infected_leaves/infected_leaves_3/'
FAST5IN_DIR = '../../data/genomic_data/infected_leaves/workspace_fast5/infected_leaves_3_fast5_out/'
OUT_DIR = '../../analyses/single_fast5s/infected_leaves/infected_leaves_3_mapped_single_fast5/'


### *****Define directories before running on different sampes*****

In [3]:
BAMIN_DIR = os.path.abspath(BAMIN_DIR)
FAST5IN_DIR = os.path.abspath(FAST5IN_DIR)
OUT_DIR = os.path.abspath(OUT_DIR)

In [4]:
##catch the bam based on naming convention
inbam_fn = [os.path.join(BAMIN_DIR, x) for x in os.listdir(BAMIN_DIR) if x.endswith('sorted.bam')][0]

In [5]:
##generated the read ID list
mappedids_fn = inbam_fn.replace('.bam', '.mappedids.txt')
#!samtools  view -F 4  {inbam_fn} | cut -f 1 | sort | uniq > {mappedids_fn}

In [6]:
!head {mappedids_fn}

00002a25-e616-429e-ab4f-b9f18e222dea
0000ac65-2942-4a3f-94c0-d1d784d0de1a
0000c55c-ea37-48e2-8bb0-84472ea6446d
0000d5dd-cd6c-4941-9928-865eac457b98
00017ecf-bb06-4bbd-81ea-4826683a64f0
00018e95-35cf-4f6e-9193-07962768a6a5
0001dbb8-3b02-49d2-b88b-0df1293088d7
0001de9a-5c62-4f31-b0ba-da3c7e5d266b
0002328a-d643-4563-b624-da289675b07a
0002dcae-475d-46a6-b8be-b79010ec5f4b


In [7]:
### generate a temporary folder that holds the batch fast5s
TMPOUT_DIR = os.path.join(OUT_DIR, 'tmp')
if not os.path.exists(TMPOUT_DIR):
    os.makedirs(TMPOUT_DIR)

In [8]:
#read in the mapped reads ids
mapped_reads = []
with open(mappedids_fn) as fh:
    for line in fh:
        mapped_reads.append(line.rstrip())

In [9]:
mapped_reads = set(mapped_reads)

In [10]:
###go from multies fast5 of mapped reads to single fast5s
!multi_to_single_fast5 -i {FAST5IN_DIR} -s {TMPOUT_DIR} -t 10 --recursive
!touch {OUT_DIR}/single_fast5.done

^C25 of 1942|                                         |  1% ETA:  1 day, 4:27:17
Traceback (most recent call last):
  File "/home/jamila/anaconda3/bin/multi_to_single_fast5", line 10, in <module>
    sys.exit(main())
  File "/home/jamila/anaconda3/lib/python3.7/site-packages/ont_fast5_api/conversion_tools/multi_to_single_fast5.py", line 104, in main
    args.recursive, follow_symlinks=not args.ignore_symlinks)
  File "/home/jamila/anaconda3/lib/python3.7/site-packages/ont_fast5_api/conversion_tools/multi_to_single_fast5.py", line 40, in batch_convert_multi_files_to_single
    pool.join()
  File "/home/jamila/anaconda3/lib/python3.7/multiprocessing/pool.py", line 556, in join
    self._worker_handler.join()
  File "/home/jamila/anaconda3/lib/python3.7/threading.py", line 1044, in join
    self._wait_for_tstate_lock()
  File "/home/jamila/anaconda3/lib/python3.7/threading.py", line 1060, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt


In [68]:
#move fast5s you want from tmp to out dir
count = 0
for directory in os.listdir(TMPOUT_DIR):
    directory = os.path.join(TMPOUT_DIR, directory)
    #check if path is directory
    if os.path.isdir(directory):
        print(directory)
        fast5s = [fn for fn in os.listdir(directory)]
        for fast5_file in fast5s:
            if fast5_file.rstrip('.fast5') in mapped_reads:
                count = count + 1
                #move the files by renaming absolute path
                old_fn = os.path.join(directory, fast5_file)
                new_fn = os.path.join(OUT_DIR, fast5_file)
                os.replace(old_fn, new_fn)
        #count = count + len(fast5s)
        print(count)
        
    #print(os.path.isfile(x))

/home/jamila/jamila_Storage/analyses/single_fast5s/germinated_spores/rep1/tmp/20
0
/home/jamila/jamila_Storage/analyses/single_fast5s/germinated_spores/rep1/tmp/6
0
/home/jamila/jamila_Storage/analyses/single_fast5s/germinated_spores/rep1/tmp/36
0
/home/jamila/jamila_Storage/analyses/single_fast5s/germinated_spores/rep1/tmp/61
0
/home/jamila/jamila_Storage/analyses/single_fast5s/germinated_spores/rep1/tmp/38
0
/home/jamila/jamila_Storage/analyses/single_fast5s/germinated_spores/rep1/tmp/90
0
/home/jamila/jamila_Storage/analyses/single_fast5s/germinated_spores/rep1/tmp/10
0
/home/jamila/jamila_Storage/analyses/single_fast5s/germinated_spores/rep1/tmp/37
0
/home/jamila/jamila_Storage/analyses/single_fast5s/germinated_spores/rep1/tmp/79
0
/home/jamila/jamila_Storage/analyses/single_fast5s/germinated_spores/rep1/tmp/83
0
/home/jamila/jamila_Storage/analyses/single_fast5s/germinated_spores/rep1/tmp/78
0
/home/jamila/jamila_Storage/analyses/single_fast5s/germinated_spores/rep1/tmp/45
0
/home

In [1]:
count == len(mapped_reads)

NameError: name 'count' is not defined

In [74]:
for x in os.listdir(OUT_DIR):
    if not x.endswith('.fast5'):
        print(x)

In [70]:
#remove tmp folder
!rm -r {TMPOUT_DIR}


In [2]:
len(mapped_reads)

NameError: name 'mapped_reads' is not defined